In [281]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [292]:
'''
agenda: 

    first 7 lines: 
        switch 1-5 to 5-1
    tomas miskov change amsterdam to kosice
    change all weird bratislavas to Bratislava
    rename col names
    
    parse numerical values in data 
    
    dump back to .csv
    

key_map:
    Bicyklovanie na / v: (pss)
        [cestách s vysokou hustotou automobilovej premávky] = pss_traffic_high_density
        [cestách s vysokou rýchlosťou premávky] = pss_traffic_high_speed
        [cestách so železničnou infraštruktúrou (napr. elekričkové trate alebo priecestia)] = pss_rail_infrastructure
        [cestách s nejasnými / mätúcimi križovatkami] = pss_confusing_xsections
        [zle osvetlených cestách alebo v tme] = pss_poorly_lit
        [cyklistických pruhoch namaľovaných na ceste] = pss_road_bike_lanes
        [cyklistických pruhoch úplne oddelených od vozovky] = pss_separate_bike_lanes
        [chodníkoch a iných cestách určených pre chodcov] = pss_pavements
        [cestách vedľa vchodov, východov a garážach] = pss_entrances_exits
        [v protismere v jednosmernej ulici] = pss_one_way_street
        [cestách so zlými povrchovými podmienkami] = pss_poor_road_conditions
        [zime (ročnom období) a snehu] = pss_winter
        [mokrých cestách a daždivom počasí] = pss_wet_roads
        [popri zaparkovaných automobiloch (ktoré môžu nepredvídateľne vojsť do cesty / otvoriť dvere)] = pss_parked_cars
    Nižšie prosím uveďte, do akej miery súhlasite s nasledovnými tvrdeniami.: (rt)
        [Pre cyklistu je prijateľné porušiť pravidlá, pokiaľ sa v danej situácii nenachádzajú žiadne iné osoby.] = rt_rule_breaking
        [Je prijateľné ísť na červenú, pokiaľ nikto iný nie je v situácii prítomný.] = rt_red_light_run
        [Je prijateľné na bicykli riskovať, pokiaľ ste riziku vystavení iba vy.] = rt_risk_taking_alone
        [Porušením pravidiel sa nutne nestávam z pohľadu bezpečnosti rizikovejším cyklistom v porovnaní s tými, ktorí pravidlá vždy dodržiavajú.] = rt_rule_breaking_relative_risk
        [Je prijateľné bicyklovať po miernom užití alkoholu (<0.5‰)] = rt_alcohol
        [Z pohľadu cyklistov mnoho dopravných pravidiel nie je možné dodržať.] = rt_rule_obeying_difficulty
        [Na bicykli používam predné svetlo vždy, keď je to potrebné.] = rt_front_light
        [Na bicykli používam zadné svetlo vždy, keď je to potrebné.] = rt_back_light
        [Počas jazdy na bicykli nosím reflexné oblečenie / oblečenie s reflexnými prvkami.] = rt_reflexive_apparell
        [Pokiaľ je to možné, pri bicyklovaní prejdem z automobilovej cesty na chodník / medzi chodcov.] = rt_road_to_pavement_switch
        [Pokiaľ je to možné, pri bicyklovaní prejdem z chodníku na automobilovú cestu.] = rt_pavement_to_road_switch
        [Radšej by som jazdil na ceste, medzi automobilmi, ako na chodníku medzi chodcami.] = rt_prefers_road
        [Mám pocit, že jazdím rýchlejšie ako ostatní cyklisti, ktorých zvyčajne vidím.] = rt_subjective_faster_riding
        [Mám pocit, že jazdím bezpečnejšie ako ostatní cyklisti, ktorých zvyčajne vidím.] = rt_subjective_safer_riding
        [V zákrutách a pri odbočovaní zvyknem používať ručné signály.] = rt_hand_signals
    Nižšie prosím uveďte, do akej miery súhlasite s nasledovnými tvrdeniami.:
        [Pri jazdení v mestskom prostredí zvyknem mať helmu.] = hw_urban_env
        [(ak relevantné) Pri jazdení v mimomestskom prostredí zvyknem mať helmu.] = hw_rural_env
        [(ak relevantné) Moja helma mi príde pohodlná.] = hw_comfort
        [(ak relevantné) Relatívne k mojim finančným možnostiam, do svojej helmy som investoval/a veľa.] = hw_financial_investment
        [Na nosení helmy mi prekážajú praktické dôvody (napr. je to ďalšia vec, ktorú treba strážiť, nezmestí sa mi do batohu či tašky, musel/a by som ju nosiť v ruke a pod.)] = hw_practical_reasons
        [Na nosení helmy mi prekáža nepohodlnosť a nepríjemnosť jej používania. (napr. nesedí mi na hlave, škrabe ma, je mi príliš veľká / malá a pod.)] = hw_comfort_reasons
        [Na nosení helmy mi prekážajú jej estetické a vizuálne prvky.] = hw_aesthetic
        [Počasie má vplyv na to, či nosím helmu.] = hw_weather_influence
'''

'\nagenda: \n\n    first 7 lines: \n        switch 1-5 to 5-1\n    tomas miskov change amsterdam to kosice\n    change all weird bratislavas to Bratislava\n    rename col names\n    \n    parse numerical values in data \n    \n    dump back to .csv\n    \n\nkey_map:\n    Bicyklovanie na / v: (pss)\n        [cestách s vysokou hustotou automobilovej premávky] = pss_traffic_high_density\n        [cestách s vysokou rýchlosťou premávky] = pss_traffic_high_speed\n        [cestách so železničnou infraštruktúrou (napr. elekričkové trate alebo priecestia)] = pss_rail_infrastructure\n        [cestách s nejasnými / mätúcimi križovatkami] = pss_confusing_xsections\n        [zle osvetlených cestách alebo v tme] = pss_poorly_lit\n        [cyklistických pruhoch namaľovaných na ceste] = pss_road_bike_lanes\n        [cyklistických pruhoch úplne oddelených od vozovky] = pss_separate_bike_lanes\n        [chodníkoch a iných cestách určených pre chodcov] = pss_pavements\n        [cestách vedľa vchodov, výc

In [293]:
filename = "dataset_final_raw"
sheet = pd.read_csv(filename + ".csv")

default_cols = {
        "Timestamp": "t",
        "Identifikujem sa ako:": "gender",
        "Vek": "age",
        "Mesto pobytu": "location",
        "Najvyšší dosiahnutý level vzdelania:": "education",
        "(dobrovoľné) Ak chcete, aby Vám boli zaslané výsledky štúdie, príp. samotná práca, môžete zanechať svoju e-mailovú adresu nižšie.  ": "email",
        "Bicykel používam najčastejšie na:": "bike_usage",
        "Primárne jazdím v prostredí:": "bike_env",
        "Bicykel používam:": "bike_freq",
        "Priemerná dĺžka jednej mojej jazdy na bicykli je: ": "bike_trip_length",
        "Subjektívne odhadujem, že ___% mojej najazdenej vzdialenosti za posledný mesiac je na cyklotrasách. ": "bike_lanes_estimate"
}

column_name_map = {
        "[cestách s vysokou hustotou automobilovej premávky]":"pss_traffic_high_density",
        "[cestách s vysokou rýchlosťou premávky]":"pss_traffic_high_speed",
        "[cestách so železničnou infraštruktúrou (napr. elekričkové trate alebo priecestia)]":"pss_rail_infrastructure",
        "[cestách s nejasnými / mätúcimi križovatkami]":"pss_confusing_xsections",
        "[zle osvetlených cestách alebo v tme]":"pss_poorly_lit",
        "[cyklistických pruhoch namaľovaných na ceste]":"pss_road_bike_lanes",
        "[cyklistických pruhoch úplne oddelených od vozovky]":"pss_separate_bike_lanes",
        "[chodníkoch a iných cestách určených pre chodcov]":"pss_pavements",
        "[cestách vedľa vchodov, východov a garážach]":"pss_entrances_exits",
        "[v protismere v jednosmernej ulici]":"pss_one_way_street",
        "[cestách so zlými povrchovými podmienkami]":"pss_poor_road_conditions",
        "[zime (ročnom období) a snehu]":"pss_winter",
        "[mokrých cestách a daždivom počasí]":"pss_wet_roads",
        "[popri zaparkovaných automobiloch (ktoré môžu nepredvídateľne vojsť do cesty / otvoriť dvere)]":"pss_parked_cars",
        "[Pre cyklistu je prijateľné porušiť pravidlá, pokiaľ sa v danej situácii nenachádzajú žiadne iné osoby.]":"rt_rule_breaking",
        "[Je prijateľné ísť na červenú, pokiaľ nikto iný nie je v situácii prítomný.]":"rt_red_light_run",
        "[Je prijateľné na bicykli riskovať, pokiaľ ste riziku vystavení iba vy.]":"rt_risk_taking_alone",
        "[Porušením pravidiel sa nutne nestávam z pohľadu bezpečnosti rizikovejším cyklistom v porovnaní s tými, ktorí pravidlá vždy dodržiavajú.]":"rt_rule_breaking_relative_risk",
        "[Je prijateľné bicyklovať po miernom užití alkoholu (<0.5‰)]":"rt_alcohol",
        "[Z pohľadu cyklistov mnoho dopravných pravidiel nie je možné dodržať.]":"rt_rule_obeying_difficulty",
        "[Na bicykli používam predné svetlo vždy, keď je to potrebné.]":"rt_front_light",
        "[Na bicykli používam zadné svetlo vždy, keď je to potrebné.]":"rt_back_light",
        "[Počas jazdy na bicykli nosím reflexné oblečenie / oblečenie s reflexnými prvkami.]":"rt_reflexive_apparell",
        "[Pokiaľ je to možné, pri bicyklovaní prejdem z automobilovej cesty na chodník / medzi chodcov.]":"rt_road_to_pavement_switch",
        "[Pokiaľ je to možné, pri bicyklovaní prejdem z chodníku na automobilovú cestu.]":"rt_pavement_to_road_switch",
        "[Radšej by som jazdil na ceste, medzi automobilmi, ako na chodníku medzi chodcami.]":"rt_prefers_road",
        "[Mám pocit, že jazdím rýchlejšie ako ostatní cyklisti, ktorých zvyčajne vidím.]":"rt_subjective_faster_riding",
        "[Mám pocit, že jazdím bezpečnejšie ako ostatní cyklisti, ktorých zvyčajne vidím.]":"rt_subjective_safer_riding",
        "[V zákrutách a pri odbočovaní zvyknem používať ručné signály.]":"rt_hand_signals",
        "[Pri jazdení v mestskom prostredí zvyknem mať helmu.]":"hw_urban_env",
        "[(ak relevantné) Pri jazdení v mimomestskom prostredí zvyknem mať helmu.]":"hw_rural_env",
        "[(ak relevantné) Moja helma mi príde pohodlná.]":"hw_comfort",
        "[(ak relevantné) Relatívne k mojim finančným možnostiam, do svojej helmy som investoval/a veľa.]":"hw_financial_investment",
        "[Na nosení helmy mi prekážajú praktické dôvody (napr. je to ďalšia vec, ktorú treba strážiť, nezmestí sa mi do batohu či tašky, musel/a by som ju nosiť v ruke a pod.)]":"hw_practical_reasons",
        "[Na nosení helmy mi prekáža nepohodlnosť a nepríjemnosť jej používania. (napr. nesedí mi na hlave, škrabe ma, je mi príliš veľká / malá a pod.)]":"hw_comfort_reasons",
        "[Na nosení helmy mi prekážajú jej estetické a vizuálne prvky.]":"hw_aesthetic",
        "[Počasie má vplyv na to, či nosím helmu.]":"hw_weather_influence",
}

In [294]:
def rename_columns_lambda(col_name):
    if col_name in default_cols:
        return default_cols[col_name]
    
    if " " in col_name and col_name != "Unnamed: 49":
        relevant = col_name[col_name.find('['):col_name.find("]")+1]
        
        for k,v in column_name_map.items():
            if relevant in k:
                print(col_name, "->", v)
                return v
                
    return col_name
    
sheet = sheet.rename(rename_columns_lambda, axis="columns")
sheet

Bicyklovanie na / v [cestách s vysokou hustotou automobilovej premávky] -> pss_traffic_high_density
Bicyklovanie na / v [cestách s vysokou rýchlosťou premávky] -> pss_traffic_high_speed
Bicyklovanie na / v [cestách so železničnou infraštruktúrou (napr. elekričkové trate alebo priecestia)] -> pss_rail_infrastructure
Bicyklovanie na / v [cestách s nejasnými / mätúcimi križovatkami] -> pss_confusing_xsections
Bicyklovanie na / v [zle osvetlených cestách alebo v tme] -> pss_poorly_lit
Bicyklovanie na / v [cyklistických pruhoch namaľovaných na ceste] -> pss_road_bike_lanes
Bicyklovanie na / v [cyklistických pruhoch úplne oddelených od vozovky] -> pss_separate_bike_lanes
Bicyklovanie na / v [chodníkoch a iných cestách určených pre chodcov] -> pss_pavements
Bicyklovanie na / v [cestách vedľa vchodov, východov a garážach] -> pss_entrances_exits
Bicyklovanie na / v [v protismere v jednosmernej ulici] -> pss_one_way_street
Bicyklovanie na / v [cestách so zlými povrchovými podmienkami] -> pss_poo

,t,gender,age,location,education,email,bike_usage,bike_env,bike_freq,bike_trip_length,bike_lanes_estimate,pss_traffic_high_density,pss_traffic_high_speed,pss_rail_infrastructure,pss_confusing_xsections,pss_poorly_lit,pss_road_bike_lanes,pss_separate_bike_lanes,pss_pavements,pss_entrances_exits,pss_one_way_street,pss_poor_road_conditions,pss_winter,pss_wet_roads,pss_parked_cars,rt_rule_breaking,rt_red_light_run,rt_risk_taking_alone,rt_rule_breaking_relative_risk,rt_alcohol,rt_rule_obeying_difficulty,rt_front_light,rt_back_light,rt_reflexive_apparell,rt_road_to_pavement_switch,rt_pavement_to_road_switch,rt_prefers_road,rt_subjective_faster_riding,rt_subjective_safer_riding,rt_hand_signals,hw_urban_env,hw_rural_env,hw_comfort,hw_financial_investment,hw_practical_reasons,hw_comfort_reasons,hw_aesthetic,hw_weather_influence,"Nižšie prosím uveďte, do akej miery súhlasite s nasledovnými tvrdeniami. [Row 9]",Unnamed: 49,"Nižšie prosím uveďte, do akej miery súhlasite s nasledovnými tvrdeniami. []"
0,3/12/2021 15:22:07,Muž,20,Bratislava,stredoškolské s maturitou,alexduni9@gmail.com,Rekreáciu,mestskom,menej ako 1x za mesiac,5 - 10 km,75,4 - považujem za nebezpečné,5 - považujem za veľmi nebezpečné,3 - neutrálny postoj,3 - neutrálny postoj,2 - považujem za bezpečné,4 - považujem za nebezpečné,1 - považujem za veľmi bezpečné,1 - považujem za veľmi bezpečné,2 - považujem za bezpečné,1 - považujem za veľmi bezpečné,1 - považujem za veľmi bezpečné,2 - považujem za bezpečné,2 - považujem za bezpečné,2 - považujem za bezpečné,5 - rozhodne súhlasím,4 - skôr súhlasím,4 - skôr súhlasím,3 - neutrálny postoj,2 - skôr nesúhlasím,4 - skôr súhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhlasím,2 - skôr nesúhlasím,4 - skôr súhlasím,2 - skôr nesúhlasím,2 - skôr nesúhlasím,2 - skôr nesúhlasím,3 - neutrálny postoj,3 - neutrálny postoj,1 - rozhodne nesúhlasím,2 - skôr nesúhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,4 - skôr súhlasím,2 - skôr nesúhlasím,NaN,NaN,NaN,NaN
1,3/15/2021 17:48:09,Muž,25,Bratislava,vysokoškolské I. stupňa (bakalárske),dan.kniebugl1@gmail.com,Transport,mestskom,5 a viac ako 5 krát za týždeň,3 - 5 km,90,4 - považujem za nebezpečné,5 - považujem za veľmi nebezpečné,4 - považujem za nebezpečné,5 - považujem za veľmi nebezpečné,3 - neutrálny postoj,4 - považujem za nebezpečné,2 - považujem za bezpečné,2 - považujem za bezpečné,4 - považujem za nebezpečné,2 - považujem za bezpečné,4 - považujem za nebezpečné,2 - považujem za bezpečné,2 - považujem za bezpečné,5 - považujem za veľmi nebezpečné,4 - skôr súhlasím,3 - neutrálny postoj,4 - skôr súhlasím,2 - skôr nesúhlasím,5 - rozhodne súhlasím,4 - skôr súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,2 - skôr nesúhlasím,4 - skôr súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,NaN,NaN,NaN
2,3/15/2021 18:15:53,Muž,20,Bratislava,stredoškolské s maturitou,bolwo77@gmail.com,Šport / tréning vo vlastnom čase,mestskom,5 a viac ako 5 krát za týždeň,20 - 30 km,5,3 - neutrálny postoj,4 - považujem za nebezpečné,3 - neutrálny postoj,3 - neutrálny postoj,4 - považujem za nebezpečné,3 - neutrálny postoj,1 - považujem za veľmi bezpečné,5 - považujem za veľmi nebezpečné,4 - považujem za nebezpečné,3 - neutrálny postoj,3 - neutrálny postoj,4 - považujem za nebezpečné,4 - považujem za nebezpečné,4 - považujem za nebezpečné,4 - skôr súhlasím,4 - skôr súhlasím,3 - neutrálny postoj,4 - skôr súhlasím,2 - skôr nesúhlasím,3 - neutrálny postoj,4 - skôr súhlasím,5 - rozhodne súhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,3 - neutrálny postoj,5 - rozhodne súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,4 - skôr súhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhl

In [295]:
switch_first_N = 6
pss_keys = [v for k,v in column_name_map.items() if v.startswith("pss")]

switch_map = {
    "5 - považujem za veľmi nebezpečné": "1 - považujem za veľmi nebezpečné",
    "4 - považujem za nebezpečné": "2 - považujem za nebezpečné",
    "3 - neutrálny postoj": "3 - neutrálny postoj",
    "2 - považujem za bezpečné": "4 - považujem za bezpečné",
    "1 - považujem za veľmi bezpečné": "5 - považujem za veľmi bezpečné"
}


for index in range(switch_first_N):
    for key in pss_keys:
        print('switching on ', sheet.loc[index,key])
        sheet.loc[index,key] = switch_map[sheet.loc[index, key]]
    
sheet.iloc[:switch_first_N]


switching on  4 - považujem za nebezpečné
switching on  5 - považujem za veľmi nebezpečné
switching on  3 - neutrálny postoj
switching on  3 - neutrálny postoj
switching on  2 - považujem za bezpečné
switching on  4 - považujem za nebezpečné
switching on  1 - považujem za veľmi bezpečné
switching on  1 - považujem za veľmi bezpečné
switching on  2 - považujem za bezpečné
switching on  1 - považujem za veľmi bezpečné
switching on  1 - považujem za veľmi bezpečné
switching on  2 - považujem za bezpečné
switching on  2 - považujem za bezpečné
switching on  2 - považujem za bezpečné
switching on  4 - považujem za nebezpečné
switching on  5 - považujem za veľmi nebezpečné
switching on  4 - považujem za nebezpečné
switching on  5 - považujem za veľmi nebezpečné
switching on  3 - neutrálny postoj
switching on  4 - považujem za nebezpečné
switching on  2 - považujem za bezpečné
switching on  2 - považujem za bezpečné
switching on  4 - považujem za nebezpečné
switching on  2 - považujem za bezp

,t,gender,age,location,education,email,bike_usage,bike_env,bike_freq,bike_trip_length,bike_lanes_estimate,pss_traffic_high_density,pss_traffic_high_speed,pss_rail_infrastructure,pss_confusing_xsections,pss_poorly_lit,pss_road_bike_lanes,pss_separate_bike_lanes,pss_pavements,pss_entrances_exits,pss_one_way_street,pss_poor_road_conditions,pss_winter,pss_wet_roads,pss_parked_cars,rt_rule_breaking,rt_red_light_run,rt_risk_taking_alone,rt_rule_breaking_relative_risk,rt_alcohol,rt_rule_obeying_difficulty,rt_front_light,rt_back_light,rt_reflexive_apparell,rt_road_to_pavement_switch,rt_pavement_to_road_switch,rt_prefers_road,rt_subjective_faster_riding,rt_subjective_safer_riding,rt_hand_signals,hw_urban_env,hw_rural_env,hw_comfort,hw_financial_investment,hw_practical_reasons,hw_comfort_reasons,hw_aesthetic,hw_weather_influence,"Nižšie prosím uveďte, do akej miery súhlasite s nasledovnými tvrdeniami. [Row 9]",Unnamed: 49,"Nižšie prosím uveďte, do akej miery súhlasite s nasledovnými tvrdeniami. []"
0,3/12/2021 15:22:07,Muž,20,Bratislava,stredoškolské s maturitou,alexduni9@gmail.com,Rekreáciu,mestskom,menej ako 1x za mesiac,5 - 10 km,75,2 - považujem za nebezpečné,1 - považujem za veľmi nebezpečné,3 - neutrálny postoj,3 - neutrálny postoj,4 - považujem za bezpečné,2 - považujem za nebezpečné,5 - považujem za veľmi bezpečné,5 - považujem za veľmi bezpečné,4 - považujem za bezpečné,5 - považujem za veľmi bezpečné,5 - považujem za veľmi bezpečné,4 - považujem za bezpečné,4 - považujem za bezpečné,4 - považujem za bezpečné,5 - rozhodne súhlasím,4 - skôr súhlasím,4 - skôr súhlasím,3 - neutrálny postoj,2 - skôr nesúhlasím,4 - skôr súhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhlasím,2 - skôr nesúhlasím,4 - skôr súhlasím,2 - skôr nesúhlasím,2 - skôr nesúhlasím,2 - skôr nesúhlasím,3 - neutrálny postoj,3 - neutrálny postoj,1 - rozhodne nesúhlasím,2 - skôr nesúhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,4 - skôr súhlasím,2 - skôr nesúhlasím,NaN,NaN,NaN,NaN
1,3/15/2021 17:48:09,Muž,25,Bratislava,vysokoškolské I. stupňa (bakalárske),dan.kniebugl1@gmail.com,Transport,mestskom,5 a viac ako 5 krát za týždeň,3 - 5 km,90,2 - považujem za nebezpečné,1 - považujem za veľmi nebezpečné,2 - považujem za nebezpečné,1 - považujem za veľmi nebezpečné,3 - neutrálny postoj,2 - považujem za nebezpečné,4 - považujem za bezpečné,4 - považujem za bezpečné,2 - považujem za nebezpečné,4 - považujem za bezpečné,2 - považujem za nebezpečné,4 - považujem za bezpečné,4 - považujem za bezpečné,1 - považujem za veľmi nebezpečné,4 - skôr súhlasím,3 - neutrálny postoj,4 - skôr súhlasím,2 - skôr nesúhlasím,5 - rozhodne súhlasím,4 - skôr súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,2 - skôr nesúhlasím,4 - skôr súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,NaN,NaN,NaN
2,3/15/2021 18:15:53,Muž,20,Bratislava,stredoškolské s maturitou,bolwo77@gmail.com,Šport / tréning vo vlastnom čase,mestskom,5 a viac ako 5 krát za týždeň,20 - 30 km,5,3 - neutrálny postoj,2 - považujem za nebezpečné,3 - neutrálny postoj,3 - neutrálny postoj,2 - považujem za nebezpečné,3 - neutrálny postoj,5 - považujem za veľmi bezpečné,1 - považujem za veľmi nebezpečné,2 - považujem za nebezpečné,3 - neutrálny postoj,3 - neutrálny postoj,2 - považujem za nebezpečné,2 - považujem za nebezpečné,2 - považujem za nebezpečné,4 - skôr súhlasím,4 - skôr súhlasím,3 - neutrálny postoj,4 - skôr súhlasím,2 - skôr nesúhlasím,3 - neutrálny postoj,4 - skôr súhlasím,5 - rozhodne súhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,3 - neutrálny postoj,5 - rozhodne súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,4 - skôr súhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhl

In [296]:
#custom rules

total_len = sheet.shape[0]

def change(_from, to):
    print(_from, "->", to)

bratislavas = ["bratislava", "ba"]
for i in range(int(total_len)):
    if sheet.loc[i, 'email'] == 'miskov.tomi@gmail.com':
        change(sheet.loc[i, 'location'], "Košice")
        sheet.loc[i, 'location'] = "Košice"
        
    change_to, loc = "Bratislava", sheet.loc[i, 'location']
    if ('bratislava' in loc.lower() or loc.lower() in bratislavas) and loc != "Bratislava" :
        change(loc, change_to)
        sheet.loc[i, 'location'] = change_to
        
sheet['email'] = sheet['email'].fillna('-')

# wtf are these
to_remove = [
    'Nižšie prosím uveďte, do akej miery súhlasite s nasledovnými tvrdeniami.  [Row 9]',
    'Unnamed: 49',
    'Nižšie prosím uveďte, do akej miery súhlasite s nasledovnými tvrdeniami.  []'
]
sheet = sheet.drop(columns=to_remove)
    
sheet

Bratislava  -> Bratislava
Bratislava  -> Bratislava
Bratislava  -> Bratislava
BRATISLAVA -> Bratislava
BA -> Bratislava
BA -> Bratislava
ba -> Bratislava
Bratislava  -> Bratislava
Bratislava  -> Bratislava
Ba -> Bratislava
Bratislava II -> Bratislava
bratislava -> Bratislava
Ba -> Bratislava
BA -> Bratislava
Bratislava  -> Bratislava
Bratislava IV -> Bratislava
bratislava -> Bratislava
Amsterdam -> Košice
bratislava -> Bratislava
Bratislava  -> Bratislava
Ba -> Bratislava
bratislava 5 -> Bratislava
Bratislava Rača -> Bratislava
Bratislava - Stare mesto -> Bratislava
Bratislava, Piešťany -> Bratislava
bratislava -> Bratislava


,t,gender,age,location,education,email,bike_usage,bike_env,bike_freq,bike_trip_length,bike_lanes_estimate,pss_traffic_high_density,pss_traffic_high_speed,pss_rail_infrastructure,pss_confusing_xsections,pss_poorly_lit,pss_road_bike_lanes,pss_separate_bike_lanes,pss_pavements,pss_entrances_exits,pss_one_way_street,pss_poor_road_conditions,pss_winter,pss_wet_roads,pss_parked_cars,rt_rule_breaking,rt_red_light_run,rt_risk_taking_alone,rt_rule_breaking_relative_risk,rt_alcohol,rt_rule_obeying_difficulty,rt_front_light,rt_back_light,rt_reflexive_apparell,rt_road_to_pavement_switch,rt_pavement_to_road_switch,rt_prefers_road,rt_subjective_faster_riding,rt_subjective_safer_riding,rt_hand_signals,hw_urban_env,hw_rural_env,hw_comfort,hw_financial_investment,hw_practical_reasons,hw_comfort_reasons,hw_aesthetic,hw_weather_influence
0,3/12/2021 15:22:07,Muž,20,Bratislava,stredoškolské s maturitou,alexduni9@gmail.com,Rekreáciu,mestskom,menej ako 1x za mesiac,5 - 10 km,75,2 - považujem za nebezpečné,1 - považujem za veľmi nebezpečné,3 - neutrálny postoj,3 - neutrálny postoj,4 - považujem za bezpečné,2 - považujem za nebezpečné,5 - považujem za veľmi bezpečné,5 - považujem za veľmi bezpečné,4 - považujem za bezpečné,5 - považujem za veľmi bezpečné,5 - považujem za veľmi bezpečné,4 - považujem za bezpečné,4 - považujem za bezpečné,4 - považujem za bezpečné,5 - rozhodne súhlasím,4 - skôr súhlasím,4 - skôr súhlasím,3 - neutrálny postoj,2 - skôr nesúhlasím,4 - skôr súhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhlasím,2 - skôr nesúhlasím,4 - skôr súhlasím,2 - skôr nesúhlasím,2 - skôr nesúhlasím,2 - skôr nesúhlasím,3 - neutrálny postoj,3 - neutrálny postoj,1 - rozhodne nesúhlasím,2 - skôr nesúhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,4 - skôr súhlasím,2 - skôr nesúhlasím,NaN
1,3/15/2021 17:48:09,Muž,25,Bratislava,vysokoškolské I. stupňa (bakalárske),dan.kniebugl1@gmail.com,Transport,mestskom,5 a viac ako 5 krát za týždeň,3 - 5 km,90,2 - považujem za nebezpečné,1 - považujem za veľmi nebezpečné,2 - považujem za nebezpečné,1 - považujem za veľmi nebezpečné,3 - neutrálny postoj,2 - považujem za nebezpečné,4 - považujem za bezpečné,4 - považujem za bezpečné,2 - považujem za nebezpečné,4 - považujem za bezpečné,2 - považujem za nebezpečné,4 - považujem za bezpečné,4 - považujem za bezpečné,1 - považujem za veľmi nebezpečné,4 - skôr súhlasím,3 - neutrálny postoj,4 - skôr súhlasím,2 - skôr nesúhlasím,5 - rozhodne súhlasím,4 - skôr súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,2 - skôr nesúhlasím,4 - skôr súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím
2,3/15/2021 18:15:53,Muž,20,Bratislava,stredoškolské s maturitou,bolwo77@gmail.com,Šport / tréning vo vlastnom čase,mestskom,5 a viac ako 5 krát za týždeň,20 - 30 km,5,3 - neutrálny postoj,2 - považujem za nebezpečné,3 - neutrálny postoj,3 - neutrálny postoj,2 - považujem za nebezpečné,3 - neutrálny postoj,5 - považujem za veľmi bezpečné,1 - považujem za veľmi nebezpečné,2 - považujem za nebezpečné,3 - neutrálny postoj,3 - neutrálny postoj,2 - považujem za nebezpečné,2 - považujem za nebezpečné,2 - považujem za nebezpečné,4 - skôr súhlasím,4 - skôr súhlasím,3 - neutrálny postoj,4 - skôr súhlasím,2 - skôr nesúhlasím,3 - neutrálny postoj,4 - skôr súhlasím,5 - rozhodne súhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,3 - neutrálny postoj,5 - rozhodne súhlasím,5 - rozhodne súhlasím,5 - rozhodne súhlasím,4 - skôr súhlasím,2 - skôr nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,1 - rozhodne nesúhlasím,5 - rozhodne súhlasím
3,3/16/2021 22:34:54,Muž,36,Bratislava,vysokoškolské II. stupňa (magisterké alebo inž...,j.t.bakos@gmail.com,Transport,mes

In [299]:
import re
relevant_cols = column_name_map.values()

def strip_non_numeric(s):
    return re.sub("[^0-9]", "", str(s))

for col in relevant_cols:
    sheet[col] = sheet[col].fillna(0)
    
for index in range(int(total_len)):
    for relevant_col in relevant_cols:
        sheet.loc[index, relevant_col] = strip_non_numeric(sheet.loc[index, relevant_col])

sheet
    

,t,gender,age,location,education,email,bike_usage,bike_env,bike_freq,bike_trip_length,bike_lanes_estimate,pss_traffic_high_density,pss_traffic_high_speed,pss_rail_infrastructure,pss_confusing_xsections,pss_poorly_lit,pss_road_bike_lanes,pss_separate_bike_lanes,pss_pavements,pss_entrances_exits,pss_one_way_street,pss_poor_road_conditions,pss_winter,pss_wet_roads,pss_parked_cars,rt_rule_breaking,rt_red_light_run,rt_risk_taking_alone,rt_rule_breaking_relative_risk,rt_alcohol,rt_rule_obeying_difficulty,rt_front_light,rt_back_light,rt_reflexive_apparell,rt_road_to_pavement_switch,rt_pavement_to_road_switch,rt_prefers_road,rt_subjective_faster_riding,rt_subjective_safer_riding,rt_hand_signals,hw_urban_env,hw_rural_env,hw_comfort,hw_financial_investment,hw_practical_reasons,hw_comfort_reasons,hw_aesthetic,hw_weather_influence
0,3/12/2021 15:22:07,Muž,20,Bratislava,stredoškolské s maturitou,alexduni9@gmail.com,Rekreáciu,mestskom,menej ako 1x za mesiac,5 - 10 km,75,2,1,3,3,4,2,5,5,4,5,5,4,4,4,5,4,4,3,2,4,2,1,2,4,2,2,2,3,3,1,2,2,1,1,4,2,0
1,3/15/2021 17:48:09,Muž,25,Bratislava,vysokoškolské I. stupňa (bakalárske),dan.kniebugl1@gmail.com,Transport,mestskom,5 a viac ako 5 krát za týždeň,3 - 5 km,90,2,1,2,1,3,2,4,4,2,4,2,4,4,1,4,3,4,2,5,4,5,5,2,1,5,5,2,4,5,5,5,5,1,1,1,1,1
2,3/15/2021 18:15:53,Muž,20,Bratislava,stredoškolské s maturitou,bolwo77@gmail.com,Šport / tréning vo vlastnom čase,mestskom,5 a viac ako 5 krát za týždeň,20 - 30 km,5,3,2,3,3,2,3,5,1,2,3,3,2,2,2,4,4,3,4,2,3,4,5,2,1,5,5,5,3,5,5,5,4,2,1,1,1,5
3,3/16/2021 22:34:54,Muž,36,Bratislava,vysokoškolské II. stupňa (magisterké alebo inž...,j.t.bakos@gmail.com,Transport,mestskom,5 a viac ako 5 krát za týždeň,5 - 10 km,25,2,2,3,3,3,4,5,4,3,4,3,3,3,3,4,2,3,4,4,4,5,5,3,3,2,3,4,3,4,5,5,5,1,1,1,1,1
4,3/16/2021 22:36:03,Muž,24,Prievidza,vysokoškolské I. stupňa (bakalárske),durta.michal@gmail.com,Transport,mestskom,5 a viac ako 5 krát za týždeň,1 - 3 km,25,2,1,2,1,2,3,5,4,2,4,2,4,4,3,4,1,3,3,4,4,4,4,2,3,4,4,1,4,5,1,1,1,1,4,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,3/26/2021 9:59:09,Muž,35,Bratislava,vysokoškolské I. stupňa (bakalárske),-,Presun/dopravu,mestskom,5 a viac ako 5-krát za týždeň,5 - 10 km,20,3,1,3,2,3,4,5,5,3,3,3,4,4,2,5,5,5,5,5,3,4,4,4,2,4,2,3,3,5,2,4,3,3,4,2,1,2
211,3/27/2021 15:33:53,Muž,43,Bratislava,vysokoškolské II. stupňa (magisterké alebo inž...,-,Presun/dopravu,mestskom,5 a viac ako 5-krát za týždeň,10 - 20 km,15,2,1,4,2,4,2,4,3,3,3,3,3,3,3,4,4,5,4,3,2,4,5,5,2,5,3,4,4,4,5,5,4,2,1,1,1,1
212,3/29/2021 18:30:02,Žena,42,Bratislava,vysokoškolské II. stupňa (magisterké alebo inž...,-,Rekreáciu,nedokážem zhodnotiť,5 a viac ako 5-krát za týždeň,20 - 30 km,5,1,1,1,1,1,1,4,2,2,2,1,1,2,1,1,1,1,3,1,3,4,4,5,4,4,2,2,3,5,4,5,4,5,1,4,4,2
213,3/29/2021 21:51:48,Žena,48,Bratislava,vysokoškolské II. stupňa (magisterké alebo inž...,jana.pucikova@gmail.com,Presun/dopravu,mestskom,5 a viac ako 5-krát za týždeň,3 - 5 km,10,3,2,4,4,2,3,5,5,2,3,3,3,3,1,5,5,2,2,2,4,2,4,1,2,3,5,1,3,5,1,1,0,0,5,5,1,1


In [300]:
anonymize = False

if anonymize:
    sheet['email'] = 'xxx@xx.com'
    
sheet

,t,gender,age,location,education,email,bike_usage,bike_env,bike_freq,bike_trip_length,bike_lanes_estimate,pss_traffic_high_density,pss_traffic_high_speed,pss_rail_infrastructure,pss_confusing_xsections,pss_poorly_lit,pss_road_bike_lanes,pss_separate_bike_lanes,pss_pavements,pss_entrances_exits,pss_one_way_street,pss_poor_road_conditions,pss_winter,pss_wet_roads,pss_parked_cars,rt_rule_breaking,rt_red_light_run,rt_risk_taking_alone,rt_rule_breaking_relative_risk,rt_alcohol,rt_rule_obeying_difficulty,rt_front_light,rt_back_light,rt_reflexive_apparell,rt_road_to_pavement_switch,rt_pavement_to_road_switch,rt_prefers_road,rt_subjective_faster_riding,rt_subjective_safer_riding,rt_hand_signals,hw_urban_env,hw_rural_env,hw_comfort,hw_financial_investment,hw_practical_reasons,hw_comfort_reasons,hw_aesthetic,hw_weather_influence
0,3/12/2021 15:22:07,Muž,20,Bratislava,stredoškolské s maturitou,alexduni9@gmail.com,Rekreáciu,mestskom,menej ako 1x za mesiac,5 - 10 km,75,2,1,3,3,4,2,5,5,4,5,5,4,4,4,5,4,4,3,2,4,2,1,2,4,2,2,2,3,3,1,2,2,1,1,4,2,0
1,3/15/2021 17:48:09,Muž,25,Bratislava,vysokoškolské I. stupňa (bakalárske),dan.kniebugl1@gmail.com,Transport,mestskom,5 a viac ako 5 krát za týždeň,3 - 5 km,90,2,1,2,1,3,2,4,4,2,4,2,4,4,1,4,3,4,2,5,4,5,5,2,1,5,5,2,4,5,5,5,5,1,1,1,1,1
2,3/15/2021 18:15:53,Muž,20,Bratislava,stredoškolské s maturitou,bolwo77@gmail.com,Šport / tréning vo vlastnom čase,mestskom,5 a viac ako 5 krát za týždeň,20 - 30 km,5,3,2,3,3,2,3,5,1,2,3,3,2,2,2,4,4,3,4,2,3,4,5,2,1,5,5,5,3,5,5,5,4,2,1,1,1,5
3,3/16/2021 22:34:54,Muž,36,Bratislava,vysokoškolské II. stupňa (magisterké alebo inž...,j.t.bakos@gmail.com,Transport,mestskom,5 a viac ako 5 krát za týždeň,5 - 10 km,25,2,2,3,3,3,4,5,4,3,4,3,3,3,3,4,2,3,4,4,4,5,5,3,3,2,3,4,3,4,5,5,5,1,1,1,1,1
4,3/16/2021 22:36:03,Muž,24,Prievidza,vysokoškolské I. stupňa (bakalárske),durta.michal@gmail.com,Transport,mestskom,5 a viac ako 5 krát za týždeň,1 - 3 km,25,2,1,2,1,2,3,5,4,2,4,2,4,4,3,4,1,3,3,4,4,4,4,2,3,4,4,1,4,5,1,1,1,1,4,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,3/26/2021 9:59:09,Muž,35,Bratislava,vysokoškolské I. stupňa (bakalárske),-,Presun/dopravu,mestskom,5 a viac ako 5-krát za týždeň,5 - 10 km,20,3,1,3,2,3,4,5,5,3,3,3,4,4,2,5,5,5,5,5,3,4,4,4,2,4,2,3,3,5,2,4,3,3,4,2,1,2
211,3/27/2021 15:33:53,Muž,43,Bratislava,vysokoškolské II. stupňa (magisterké alebo inž...,-,Presun/dopravu,mestskom,5 a viac ako 5-krát za týždeň,10 - 20 km,15,2,1,4,2,4,2,4,3,3,3,3,3,3,3,4,4,5,4,3,2,4,5,5,2,5,3,4,4,4,5,5,4,2,1,1,1,1
212,3/29/2021 18:30:02,Žena,42,Bratislava,vysokoškolské II. stupňa (magisterké alebo inž...,-,Rekreáciu,nedokážem zhodnotiť,5 a viac ako 5-krát za týždeň,20 - 30 km,5,1,1,1,1,1,1,4,2,2,2,1,1,2,1,1,1,1,3,1,3,4,4,5,4,4,2,2,3,5,4,5,4,5,1,4,4,2
213,3/29/2021 21:51:48,Žena,48,Bratislava,vysokoškolské II. stupňa (magisterké alebo inž...,jana.pucikova@gmail.com,Presun/dopravu,mestskom,5 a viac ako 5-krát za týždeň,3 - 5 km,10,3,2,4,4,2,3,5,5,2,3,3,3,3,1,5,5,2,2,2,4,2,4,1,2,3,5,1,3,5,1,1,0,0,5,5,1,1


In [301]:
out_filename = filename + "_transformed"
sheet.to_csv(out_filename + ".csv")

In [302]:
relevant_cols

dict_values(['pss_traffic_high_density', 'pss_traffic_high_speed', 'pss_rail_infrastructure', 'pss_confusing_xsections', 'pss_poorly_lit', 'pss_road_bike_lanes', 'pss_separate_bike_lanes', 'pss_pavements', 'pss_entrances_exits', 'pss_one_way_street', 'pss_poor_road_conditions', 'pss_winter', 'pss_wet_roads', 'pss_parked_cars', 'rt_rule_breaking', 'rt_red_light_run', 'rt_risk_taking_alone', 'rt_rule_breaking_relative_risk', 'rt_alcohol', 'rt_rule_obeying_difficulty', 'rt_front_light', 'rt_back_light', 'rt_reflexive_apparell', 'rt_road_to_pavement_switch', 'rt_pavement_to_road_switch', 'rt_prefers_road', 'rt_subjective_faster_riding', 'rt_subjective_safer_riding', 'rt_hand_signals', 'hw_urban_env', 'hw_rural_env', 'hw_comfort', 'hw_financial_investment', 'hw_practical_reasons', 'hw_comfort_reasons', 'hw_aesthetic', 'hw_weather_influence'])